此文件包含了对sd数据（来自公司员工的短信息交流数据）的coinrate 的计算

In [ ]:
%matplotlib auto
#弹出窗绘图

In [ ]:
%matplotlib inline
#控制台绘图

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation

import pandas as pd
import numpy as np
import numba as nb
import networkx as nx

from scipy import stats
from mpl_toolkits.mplot3d import Axes3D

import random
import string
import time


---

## 运行计算函数

导入函数

In [ ]:
%%time
#%%prun

%run ./coin_rate_calculate.ipynb
%run ./picture_by_matplotlib.ipynb

%run ./ 当前目录

%run ../ 上一级目录

---

## 分析sd数据

### 读取数据

分析文献中的sd数据（来自公司员工的短信息交流数据）

In [ ]:
pd.set_option('display.max_columns', 10,'display.max_rows', 10)  # 设置显示全部列数据行数据

In [ ]:
# 读取数据
colname=['sender','recipient','timestamp']
datel=pd.read_table('.\\data\\SD01.txt',sep='\s+',header=None,names=colname,nrows=None)

In [ ]:
# 获取活跃的用户的信息，发送信息大于一定值的用户
unknow=pd.value_counts(datel.loc[:,'sender'])
indx_3=unknow[unknow>500].index
print('num=%d\n'%len(indx_3),indx_3)

---

In [ ]:
temp1=datel[datel.sender==136].loc[:,'timestamp']
temp2=datel[datel.sender==135].loc[:,'timestamp']
pre_cr_(600,0,temp1,temp2)

In [ ]:
%%time
pre_cr_(600,0,temp1,temp2)

In [ ]:
%%time
pre_coin_r(600,0,temp1,temp2)

In [ ]:
pre_cr_center(600,0,temp1,temp2)

---

### 计算coinrate

In [ ]:
# 生成二维空表格
neta=pd.DataFrame(np.zeros((len(indx_3),len(indx_3))),index=indx_3,columns=indx_3)
neta

In [ ]:
# 二维数据中，行对象作为列对象的pre来计算,行对象与列对象的关联占行对象的比例
# 将计算得到的coinrate填充到二维空表格

for ind in indx_3:
    for col in indx_3:
        if ind==col:
            continue
        tp1=datel[datel.sender==ind].loc[:,'timestamp']
        tp2=datel[datel.sender==col].loc[:,'timestamp']
        neta.loc[ind,col]=pre_cr_(600,0,tp1,tp2)

In [ ]:
# 更改表格字体颜色的准备
def showcolor(val):
    if val>0.5:
        color='red'
    elif val>0.2:
        color='green'
    else :
        color='blank'
    return 'color:%s'%color

In [ ]:
# 存储变量到其他文件中也可使用
new=neta.round(2)
%store new
%store indx_3

In [ ]:
# 更改表格的颜色并展示
neta.round(decimals=2).style.applymap(showcolor)

---

### 直接通过数据计算

In [ ]:
# 直接从数据中的收发信人数据计算关系
def send_dt(sd):
    '''发信人sd，返回信息发送情况字典：sender ；rc(收信人及收信的比例)
    '''
#     发信人sd的收信人列表
    showz=datel[datel.sender==sd].loc[:,'recipient']
#     计算各个收信人的收信数量
    showzc=pd.value_counts(showz)
    rate=showzc.iloc[0]/showz.count()  #计算sender发送的信息中发给的最多的人的信息占比
    mro=showzc.index[0]
    sed_rc={'sender':sd,'rc':(showzc/showz.count())}
#     print('sender is %5d and max rate object %5d is %.3f'%(sd,mro,rate),mro in indx_3)
    return sed_rc
    

In [ ]:
# 展示所有发信人联系最密切的对象
for i in indx_3:
    temp=send_dt(i)
    print(temp)

---

### 计算coinrate得出的社交网络图

#### 设置节点和边

用箭头表示联系的流向，例如：a--->b，意味着a的信息大多发给了b

In [ ]:
# 挑选coinrate的最大值
max_col_idx=np.argmax(np.array(neta),axis=0)

# 对最大值连边
max_list_edges=[]
for i in range(len(indx_3)):
    max_list_edges.append((indx_3[i],indx_3[max_col_idx[i]]))


In [ ]:
print(data_5)
print(max_list_edges)

---

In [ ]:
# 挑选大于指定数值的数的位置，第一个数是行，第二个数是列
data_5=np.where(neta>0.5)

# 对指定的数设置连边
da5_edge=[]
for i in range(len(data_5[0])):
    x1=data_5[0][i]
    x2=data_5[1][i]
    da5_edge.append((indx_3[x1],indx_3[x2]))
    print(x1,x2,'\tindex:\t',indx_3[x1],indx_3[x2])
print(da5_edge)

In [ ]:
# 挑选大于指定数值的数的位置，第一个数是行，第二个数是列
data_5=np.where(neta>0.5)

# 对指定的数设置连边
da5_edge_re=[]
for i in range(len(data_5[0])):
    x1=data_5[0][i]
    x2=data_5[1][i]
    if x2==0 or x2==1 : continue
    da5_edge_re.append((indx_3[x1],indx_3[x2]))
    print(x1,x2,'\tindex:\t',indx_3[x1],indx_3[x2])
print(da5_edge_re)

# 设置135和136仅有一条连边
da5_edge_re.append((135,136))
da5_edge_re.append((136,135))

---

#### 画出有向网络图

In [ ]:
# 创建有向网络图，并添加节点和连边

sd_g=nx.DiGraph()
sd_g.add_nodes_from(indx_3)
sd_g.add_edges_from(da5_edge)
# sd_g.add_edges_from(max_list_edges)

In [ ]:
# 这个是来自SD01的数据，挑选出了发送信息数大于一定值的用户,绘制出了网络图

# 支持中文
plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号
# 有中文出现，需要用u'内容'

plt.figure(figsize=(20,20))
# text=u'节点是发送信息数大于500的用户,\n用户数是%d,coinrate>0.5就连边'%(len(indx_3))
text=u'节点是发送信息数大于500的用户,\n用户数是%d,coinrate>0.5,且去除了多连边'%(len(indx_3))

nx.draw(sd_g,
        with_labels=True,
        node_color='SpringGreen',
        edge_color='royalblue',
        node_size=500,
        width=1.5,
        arrowstyle='->',
        pos=nx.circular_layout(sd_g),        
#         pos=nx.kamada_kawai_layout(sd_g),
        )
# plt.title('network')
plt.text(0,0,text,fontdict={'color':'orangered','size':20})
# plt.savefig('.\\data\\network_SD01_u500_c_by_circular_.5.png')
# plt.savefig('.\\data\\network_SD01_u500_c_by_circular_re.png')
plt.show()

---

### 通过数据得出的社交网络图

In [ ]:
# 设置节点的连边，对数据的发信和收信人直接统计，若收信人也在节点中则添加该边

edge_lists=[]
for sd in indx_3:
    senre=send_dt(sd)['rc']
    filt_d=senre[senre>0.5]
    for i in range(len(filt_d)):
        obj=filt_d.index[i]
        if obj in indx_3:
            edge_lists.append((sd,obj))
#         print(filt_d.index[i],filt_d.iloc[i])
#         print('next:\n')

In [ ]:
# 创建有向网络图，并添加节点和连边

sd_og=nx.DiGraph()
sd_og.add_nodes_from(indx_3)
sd_og.add_edges_from(edge_lists)

In [ ]:
# 这个是来自SD01的数据，挑选出了发送信息数大于一定值的用户,绘制出了网络图

# 支持中文
plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号
# 有中文出现，需要用u'内容'

plt.figure(figsize=(20,20))
text=u'节点是发送信息数大于500的用户,\n用户数是{},接受了对方50%以上的信息的连边'.format(len(indx_3))

nx.draw(sd_og,
        with_labels=True,
        node_color='SpringGreen',
        edge_color='royalblue',
        node_size=500,
        width=1.5,
        arrowstyle='->',
        pos=nx.circular_layout(sd_og),        
#         pos=nx.kamada_kawai_layout(sd_og),
        )
# plt.title('network')
plt.text(0,0,text,fontdict={'color':'orangered','size':20})
# plt.savefig('.\\data\\network_SD01_u500_data_by_circular.png')
plt.show()

---

### 绘制网络的函数

In [ ]:
# u'节点是发送信息数大于500的用户,\n用户数是{},接受了对方50%以上的信息的连边'.format(len(indx_3))
# path='.\\data\\'+?

def pic_network(pot_no,edge_lists,txt_p=u'节点网络图',path_pic=None):
    '''绘制网络图；
    输入参数为：节点、连边；可选文字、保存路径
    '''
    # 创建有向网络图，并添加节点和连边

    sd_og=nx.DiGraph()
    sd_og.add_nodes_from(pot_no)
    sd_og.add_edges_from(edge_lists)

    # 支持中文
    plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
    plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号
    # 有中文出现，需要用u'内容'

    plt.figure(figsize=(20,20))

    nx.draw(sd_og,
            with_labels=True,
            node_color='SpringGreen',
            edge_color='royalblue',
            node_size=500,
            width=1.5,
            arrowstyle='->',
            pos=nx.circular_layout(sd_og),        
    #         pos=nx.kamada_kawai_layout(sd_og),
            )
    # plt.title('network')
    plt.text(0,0,txt_p,fontdict={'color':'orangered','size':20})
    if not path_pic==None:
        plt.savefig(path_pic)
    plt.show()

---

### 绘图

In [ ]:
# 设置节点的连边，对数据的发信和收信人直接统计，若收信人也在节点中则添加该边
# 挑选了收信人中最大比例的对象

max_edge_lists=[]
for sd in indx_3:
    senre=send_dt(sd)['rc']
#     filt_d=senre[senre>0.5]
    obj=senre.index[0]
    if obj in indx_3:
        max_edge_lists.append((sd,obj))
        print((sd,obj))
        print('next:\n')

In [ ]:
txt=u'节点是发送信息数大于500的用户,\n用户数是{},接受了最多的信息的对象连边'.format(len(indx_3))
path='.\\data\\network_SD01_u500_data_by_circular_max.png'
pic_network(indx_3,edge_lists,txt_p=txt,path_pic=path)

---

### 网络的相似度

In [ ]:
def similar_edge(e1,e2):
    '''计算两个连边集合的相似对度，输入两个连边的列表，输出(a&b)/(a|b)
    '''
    a1=set(e1)
    b1=set(e2)
    return len(a1 & b1)/len(a1 | b1)

In [ ]:
print(len(da5_edge),'\n',da5_edge)
print(len(max_list_edges),'\n',max_list_edges)

In [ ]:
print(len(edge_lists),'\n',edge_lists)
print(len(max_edge_lists),'\n',max_edge_lists)

In [ ]:
a1=set(da5_edge)
a2=set(max_list_edges)
a3=set(da5_edge_re)
b1=set(edge_lists)
b2=set(max_edge_lists)

In [ ]:
print(a1,'\n',a2,'\n',a3,'\n',b1,'\n',b2,'\n')

In [ ]:
cr={'cr.5':da5_edge,'cr_max':max_list_edges,'cr.5_re':da5_edge_re}
da={'data.5':edge_lists,'data_max':max_edge_lists}

In [ ]:
df=pd.DataFrame(np.zeros((3,2)),index=list(cr),columns=list(da))

In [ ]:
for ind in list(cr):
    for col in list(da):
        df.loc[ind,col]=similar_edge(cr[ind],da[col])

In [ ]:
all_data=dict(cr,**da)

In [ ]:
ndf=pd.DataFrame(np.zeros((5,5)),index=list(all_data),columns=list(all_data))

In [ ]:
for ind in list(all_data):
    for col in list(all_data):
#         if ind == col:
#             continue
        ndf.loc[ind,col]=similar_edge(all_data[ind],all_data[col])

In [ ]:
ndf.round(decimals=2)